In [ ]:
#hide
%load_ext autoreload
%autoreload 2

!pip install -r ../requirements.txt > /dev/null

In [ ]:
# default_exp dialog_system

In [ ]:

# export
from deeppavlov import configs,build_model,train_model
import json
from os import path,popen,mkdir
from shutil import copyfile
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from collections import defaultdict 

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%I:%M:%S')
logging.info("Hello! Welcome to our automated dialog system!")
logging.debug(" Debug Mode Activa")
logging.error(' Error Log Active ')

04:37:56 ERROR: Error Log Active 


In [ ]:
#export
def run_shell_installs():
    ''' Run install commands
    '''
    command_strings = (
        ' pip install deeppavlov',
        ' python -m deeppavlov install squad',
        ' python -m deeppavlov install squad_bert',
        ' python -m deeppavlov install fasttext_avg_autofaq',
        ' python -m deeppavlov install fasttext_tfidf_autofaq',
        ' python -m deeppavlov install tfidf_autofaq',
        ' python -m deeppavlov install tfidf_logreg_autofaq ',
        ' python -m deeppavlov install tfidf_logreg_en_faq'
    )
    for command in command_strings:
        logging.debug(command)
        logging.debug(popen(command).read())
    

# Dialog System
> Question Answering Automated Dialog System

In [ ]:
run_shell_installs()

In [ ]:
#export
def action_over_list_f(arr, v):

    k_id, v_id = next(iter(v[0].items()))

    for p, a in enumerate(arr):
        if k_id in a.keys() and a[k_id] == v_id:
            for k_rep, v_rep in v[1].items():
                arr[p][k_rep] = v_rep


def replacement_f(model_config, **args):
    '''Replaces the model config dictionary with new values
    '''
    for k, v in args.items():
        if isinstance(v, dict):
            replacement_f(model_config[k], **v)
        if isinstance(v, str):
            model_config[k] = v
        if isinstance(model_config[k], list):
            action_over_list_f(model_config[k], v)

In [ ]:
# test action_over_list_f
from random import randint


def gen_list_keys_for_tests():

    str_n = lambda x: f'{x}_{randint(1,10):1}'
    gen_dict_list = lambda: {
        'id': str_n('id'),
        'key1': str_n('v1'),
        'key2': str_n('v2'),
        'key3': str_n('v3')
    }

    pipe_list = [gen_dict_list() for _ in range(randint(3, 10))]

    rand_id = pipe_list[randint(0, len(pipe_list) - 1)]['id']
    rand_key = f'key{randint(1, 3)}' 

    new_rand_val = str_n('new')
    args = {
        'chains': {
            'pipe': [{
                'id': rand_id
            }, {
               rand_key : new_rand_val
            }]
        }
    }

    return pipe_list, rand_id, rand_key, args, new_rand_val


def test_action_over_list_f():

    pipe_list, rand_id, rand_key, args, new_rand_val = gen_list_keys_for_tests()

    assert all(
        new_rand_val not in pipe_elem.values() for pipe_elem in pipe_list
    )

    action_over_list_f(pipe_list, args['chains']['pipe'])

    assert any(
        rand_key in pipe_elem.keys() and
        new_rand_val in pipe_elem.values() for pipe_elem in pipe_list
    )


def test_replacement_f_list():

    pipe_list, rand_id, rand_key, args, new_rand_val = gen_list_keys_for_tests()

    mod_conf = {'chains': {'pipe': pipe_list}}

    assert all(
        new_rand_val not in pipe_elem.values()
        for pipe_elem in mod_conf['chains']['pipe']
    )

    replacement_f(model_config=mod_conf, **args)
    assert any(
        rand_key in pipe_elem.keys() and
        new_rand_val in pipe_elem.values()
        for pipe_elem in mod_conf['chains']['pipe']
    )


def test_replacement_f_val():
    args = {'key3': 'newvalue'}
    mod_conf = {'key1': 'val1', 'key2': 'val2', 'key3': 'val3'}
    replacement_f(model_config=mod_conf, **args)
    assert all(
        arg_k in mod_conf.keys() and arg_v in mod_conf.values()
        for arg_k, arg_v in args.items()
    )


def test_replacement_f_dict():
    args = {'1_key_3': {'2_key_2': 'newvalue'}}
    mod_conf = {'1_key_3': {'2_key_2': 'oldvalue'}, '0_key_': '0_val'}
    replacement_f(model_config=mod_conf, **args)
    assert mod_conf['1_key_3']['2_key_2'] == 'newvalue'


test_action_over_list_f()
test_replacement_f_list()
test_replacement_f_val()
test_replacement_f_dict()

In [ ]:
#export
def updates_faq_config_file(
    configs_path,
    **args
):
    '''Updates deepplavov json config file 
    '''
    #set FAQ data in config file
    model_config = json.load(open(configs_path))

    if 'data_url' in model_config['dataset_reader']:
        del model_config['dataset_reader']['data_url']

    replacement_f(model_config=model_config,**args)

    json.dump(model_config, open(configs_path, 'w'))

In [ ]:
#test updates_faq_config_file
import tempfile
from shutil import copyfile


def gen_list_keys_for_tests():

    str_n = lambda x: f'{x}_{randint(1,10):1}'
    gen_dict_list = lambda: {
        'id': str_n('id'),
        'key1': str_n('v1'),
        'key2': str_n('v2'),
        'key3': str_n('v3')
    }

    pipe_list = [gen_dict_list() for _ in range(randint(3, 10))]

    rand_id = pipe_list[randint(0, len(pipe_list) - 1)]['id']
    rand_key =  f'key{randint(1, 3)}' 

    new_rand_val = str_n('new')
    pipe_dict = {'pipe': [{'id': rand_id}, {rand_key: new_rand_val}]}
    args = {'chainer': pipe_dict}

    return pipe_list, rand_id, rand_key, args, new_rand_val


def test_updates_faq_config_file_update_string():

    with tempfile.TemporaryDirectory() as tmpdirname:

        tmp_config_file = path.join(tmpdirname, 'tmp_file.json')

        copyfile(configs.faq.tfidf_logreg_en_faq, tmp_config_file)

        assert path.isfile(tmp_config_file)

        updates_faq_config_file(
            configs_path=tmp_config_file,
            dataset_reader={'data_path': 'fictional_csv_file.csv'}
        )

        config_json = json.load(open(tmp_config_file))
        assert 'data_path' in config_json['dataset_reader']


def test_updates_faq_config_file_update_list():

    with tempfile.TemporaryDirectory() as tmpdirname:

        tmp_config_file = path.join(tmpdirname, 'tmp_file.json')

        pipe_list, rand_id, rand_key, args, new_rand_val = gen_list_keys_for_tests(
        )
        mod_conf = {
            'chainer': {
                'pipe': pipe_list
            },
            'dataset_reader': 'dataset_reader_dictionary'
        }

        json.dump(mod_conf, open(tmp_config_file, 'w'))

        assert path.isfile(tmp_config_file)

        updates_faq_config_file(configs_path=tmp_config_file, **args)

        config_json = json.load(open(tmp_config_file))
   
        assert any(
            rand_key in pipe_elem.keys() and new_rand_val in pipe_elem.values()
            for pipe_elem in config_json['chainer']['pipe']
        )


test_updates_faq_config_file_update_string()
test_updates_faq_config_file_update_list()

In [ ]:
#export
def select_faq_responses(faq_model, question):
    '''Calls Deeppavlov FAQ model
    '''
    return faq_model([question])[0]

In [ ]:
#test faq responses
import tempfile
from shutil import copyfile


def gen_mock_csv_file(tmpdirname, faqs):

    temp_faq_csv = path.join(tmpdirname, 'tmp_faq.csv')

    pd.DataFrame(faqs).to_csv(temp_faq_csv, index=False)

    return temp_faq_csv


def gen_mock_vocab_answers(tmpdirname, vocabs):

    temp_dict_file = path.join(tmpdirname, 'temp_vocab_answers.dict')
    vocabs_text = '\n'.join(
        t + '\t' + str(f) for t, f in zip(vocabs['text'], vocabs['freq'])
    )

    f = open(temp_dict_file, 'w')
    f.write(vocabs_text)
    f.close()

    return temp_dict_file


def gen_faq_config(tmpdirname, vocab_file, faq_file):

    temp_configs_faq = path.join(tmpdirname, 'temp_config_faq.json')
    copyfile(configs.faq.tfidf_logreg_en_faq, temp_configs_faq)

    changes_dict = {'save_path': vocab_file, 'load_path': vocab_file}
    id_dict = {'id': 'answers_vocab'}

    updates_faq_config_file(
        configs_path=temp_configs_faq,
        chainer={'pipe': [id_dict, changes_dict]},
        dataset_reader={'data_path': faq_file}
    )

    return temp_configs_faq


def test_faq_response_with_minimum_faqs_in_dataframe_fail_case():
    with tempfile.TemporaryDirectory() as tmpdirname:

        faqs = {
            'Question': ['Is Covid erradicated?'],
            'Answer': ['Definitely not!']
        }

        vocabs = {'text': ['This is a vocab example'], 'freq': [1]}

        faq_file = gen_mock_csv_file(tmpdirname, faqs)
        vocab_file = gen_mock_vocab_answers(tmpdirname, vocabs)

        configs_file = gen_faq_config(tmpdirname, vocab_file, faq_file)

        try:
            select_faq_responses(
                question='Is Enrique the prettiest person in town?',
                faq_model=train_model(configs_file, download=True)
            )
            assert False
        except ValueError as e:
            assert True


def test_faq_response_with_minimum_faqs_in_dataframe_success_case():
    with tempfile.TemporaryDirectory() as tmpdirname:

        faqs = {
            'Question': ['Is Covid erradicated?', 'Who is the current POTUS?'],
            'Answer': ['Definitely not!', 'Donald Trump']
        }

        vocabs = {'text': ['This is a vocab example'], 'freq': [1]}

        faq_file = gen_mock_csv_file(tmpdirname, faqs)
        vocab_file = gen_mock_vocab_answers(tmpdirname, vocabs)

        configs_file = gen_faq_config(tmpdirname, vocab_file, faq_file)

        assert select_faq_responses(
            question='Is Enrique the prettiest person in town?',
            faq_model=train_model(configs_file, download=True)
        ) == ['Donald Trump']

        
        
def test_faq_response_with_minimum_answers_vocab_success_case():
    with tempfile.TemporaryDirectory() as tmpdirname:

        faqs = {
            'Question': ['Is Covid erradicated?', 'Who is the current POTUS?'],
            'Answer': ['Definitely not!', 'Donald Trump']
        }

        vocabs = {'text': [], 'freq': []}

        faq_file = gen_mock_csv_file(tmpdirname, faqs)
        vocab_file = gen_mock_vocab_answers(tmpdirname, vocabs)

        configs_file = gen_faq_config(tmpdirname, vocab_file, faq_file)

        select_faq_responses(
            question='Is Enrique the prettiest person in town?',
            faq_model=train_model(configs_file, download=True)
        ) == ['Donald Trump']

test_faq_response_with_minimum_faqs_in_dataframe_fail_case()
test_faq_response_with_minimum_faqs_in_dataframe_success_case()
test_faq_response_with_minimum_answers_vocab_success_case()

2020-06-18 16:40:05.480 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/faq/mipt/en_mipt_faq_v4.tar.gz to /home/jovyan/.deeppavlov/models/faq/en_mipt_faq_v4.tar.gz
04:40:05 INFO:Downloading from http://files.deeppavlov.ai/faq/mipt/en_mipt_faq_v4.tar.gz to /home/jovyan/.deeppavlov/models/faq/en_mipt_faq_v4.tar.gz
100%|██████████| 12.3k/12.3k [00:00<00:00, 10.8MB/s]
2020-06-18 16:40:05.496 INFO in 'deeppavlov.core.data.utils'['utils'] at line 242: Extracting /home/jovyan/.deeppavlov/models/faq/en_mipt_faq_v4.tar.gz archive into /home/jovyan/.deeppavlov/models/faq/mipt
04:40:05 INFO:Extracting /home/jovyan/.deeppavlov/models/faq/en_mipt_faq_v4.tar.gz archive into /home/jovyan/.deeppavlov/models/faq/mipt
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk

In [ ]:
#export
def select_squad_responses(
    contexts, squad_model, question, best_results=1
):
    '''Calls Deeppavlov BERT and RNET Context Question Answering
    '''
    responses = contexts.context.apply(
        lambda context: squad_model([context], [question])
    ).values
    
    logging.debug(f'Responses: {responses}')
    top_responses = [
        r[0][0] for r in sorted(responses, key=lambda x: -1 * x[2][0])
        [:best_results]
    ]

    logging.debug(f'Top Responses: {top_responses}')
    return responses, top_responses

In [ ]:
#test select_squad_responses
import tempfile
from shutil import copyfile

empty = {'topic': [], 'context': []}
spacex = {
    'topic': ['SpaceX'],
    'context':
        [
            '''Space Exploration Technologies Corp., trading as SpaceX, is an American aerospace manufacturer and space transportation
services company headquartered in Hawthorne, California. It was founded in 2002 by Elon Musk with the goal of reducing space 
transportation costs to enable the colonization of Mars. SpaceX has developed several launch vehicles, the Starlink satellite
constellation, and the Dragon spacecraft. It is widely considered among the most successful private spaceflight companies.'''
        ]
}

intekglobal = {
    'topic': ['Intekglobal', 'InG'],
    'context':
        [
            'Intekglobal has its headquarters located in TJ',
            'Intekglobal is in the north of mexico'
        ]
}


def assert_squad_model(
    contexts, squad_model, question, expected_responses, **args
):
    responses, top_responses = select_squad_responses(
        contexts=pd.DataFrame(contexts),
        squad_model=squad_model,
        question=question,
        **args
    )
    assert top_responses == expected_responses


def test_squad_bert():

    bert = build_model(configs.squad.squad_bert, download=True)

    assert_squad_model(
        empty,
        bert,
        'Is an empty response expected?',
        expected_responses=[],
        best_results=2
    )

    assert_squad_model(
        spacex, bert, 'Who founded SpaceX?', expected_responses=['Elon Musk']
    )

    assert_squad_model(
        intekglobal,
        bert,
        'Where is Intekglobal located?',
        expected_responses=['north of mexico','TJ'],
        best_results=2
    )


def test_squad_rnet():

    bert = build_model(configs.squad.squad, download=True)

    assert_squad_model(
        empty,
        bert,
        'Is an empty response expected?',
        expected_responses=[],
        best_results=5
    )

    assert_squad_model(
        spacex, bert, 'Who founded SpaceX?', expected_responses=['Elon Musk']
    )

    assert_squad_model(
        intekglobal,
        bert,
        'Where is Intekglobal located?',
        expected_responses=['north of mexico','TJ'],
        best_results=2
    )

test_squad_bert()
test_squad_rnet()
del spacex, empty, intekglobal

2020-06-18 16:40:20.504 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/cased_L-12_H-768_A-12.zip to /home/jovyan/.deeppavlov/downloads/cased_L-12_H-768_A-12.zip
04:40:20 INFO:Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/cased_L-12_H-768_A-12.zip to /home/jovyan/.deeppavlov/downloads/cased_L-12_H-768_A-12.zip
 85%|████████▍ | 343M/404M [01:35<00:13, 4.71MB/s] 

In [ ]:
#export
def load_qa_models(
    config_rnet=configs.squad.squad,
    config_bert=configs.squad.squad_bert,
    config_tfidf=configs.faq.tfidf_logreg_en_faq,
    download=True
):
    qa_models = {
        'squad':
            {
                'rnet': build_model(config_rnet, download=download),
                'bert': build_model(config_bert, download=download)
            },
        'faq': {
            'tfidf': train_model(config_tfidf, download=download)
        }
    }
    return qa_models


def format_responses(question, responses):
    formatted_response = f'{question}:\n\n'
    for k, res in enumerate(responses):
        formatted_response += f'{k}: {res}\n'
    return formatted_response


def get_responses(contexts, question, qa_models, nb_squad_results=1):
    responses = []
    for squad_model in qa_models['squad'].values():
        responses.extend(
            select_squad_responses(
                contexts, squad_model, question, best_results=nb_squad_results
            )[1]
        )
    for faq_model in qa_models['faq'].values():
        responses.extend(select_faq_responses(faq_model, question))
    return responses, format_responses(
        question, set([r for r in responses if r.strip()])
    )

In [ ]:
# test get_responses
import tempfile
from shutil import copyfile

intekglobal_context = {
    'topic': ['Intekglobal', 'InG'],
    'context':
        [
            'Intekglobal has its headquarters located in TJ',
            'Intekglobal is in the north of mexico'
        ]
}

intekglobal_faqs = {
    'Question': ['Is Intekglobal an IT company?', 'Where can I apply?'],
    'Answer':
        ['Yes it is!', 'Please refer the our website for further information']
}


def mock_faq_files(tmpdirname, faqs):

    faq_files = {
        'data': path.join(tmpdirname, 'temp_faq.csv'),
        'config': path.join(tmpdirname, 'temp_config_faq.json')
    }

    pd.DataFrame(faqs).to_csv(faq_files['data'], index=False)
    copyfile(configs.faq.tfidf_logreg_en_faq, faq_files['config'])

    updates_faq_config_file(
        configs_path=faq_files['config'],
        dataset_reader={'data_path': faq_files['data']}
    )

    return faq_files


def test_get_intekglobal_responses():
    with tempfile.TemporaryDirectory() as tmpdirname:

        faq_files = mock_faq_files(tmpdirname, intekglobal_faqs)
        qa_models = load_qa_models(
            config_tfidf=faq_files['config'], download=False
        )

        responses, format_responses = get_responses(
            pd.DataFrame(intekglobal_context),
            'Where is Intekglobal?',
            qa_models,
            nb_squad_results=2
        )

        logging.debug(responses)
        logging.debug(format_responses)
        assert all(
            response in ('north of mexico', 'TJ', 'Yes it is!')
            for response in responses
        )
        #assert ''' Where is Intekglobal?:

#0: north of mexico
#1: Yes it is!
#2: TJ
#        '''.strip() == format_responses.strip()


def test_get_responses_with_empty_context():
    with tempfile.TemporaryDirectory() as tmpdirname:
        min_faqs = {'Question': ['Minimum number of questions?','This is the other question?'], 'Answer': ['Two','yes']}
        faq_files = mock_faq_files(tmpdirname, min_faqs)

        qa_models = load_qa_models(
            config_tfidf=faq_files['config'], download=False
        )
        empty_context = {'topic': [], 'context': []}

        responses, format_responses = get_responses(
            pd.DataFrame(empty_context),
            'What is the minimun number of FAQ questions',
            qa_models,
            nb_squad_results=2
        )

        logging.debug(responses)
        logging.debug(format_responses)
        assert responses == ['Two']


test_get_intekglobal_responses()
test_get_responses_with_empty_context()

del intekglobal_context

In [ ]:
#export
def get_input(text):
    '''This redundancy is needed for testing'''
    return input(text)


def new_answer(question, data, qa_models):

    if get_input('Give a better answer [y/n]?')[0].lower() != 'y':
        return 'no data updates..'

    if get_input('Give the answer as a context [y/n]?')[0].lower() == 'y':
        new_context = pd.DataFrame(
            {
                'topic': [get_input('Give context a title:\n')],
                'context': [get_input('Introduce the context:\n')]
            }
        )
        data['context']['df'] = data['context']['df'].append(new_context)
        data['context']['df'].to_csv(data['context']['path'], index=False)

        return 'contexts dataset updated..'
    else:
        new_faq = pd.DataFrame(
            {
                'Question': [question],
                'Answer': [get_input('Introduce the answer:\n')]
            }
        )
        data['faq']['df'] = data['faq']['df'].append(new_faq)
        data['faq']['df'].to_csv(data['faq']['path'], index=False)
        qa_models['faq']['tfidf'] = train_model(
            data['faq']['config'], download=False
        )
        return 'FAQ dataset and model updated..'


def question_response(data, qa_models, num_returned_values_per_squad_model=1):
    question = get_input('Introduce question:\n')

    _, formatted_responses = get_responses(
        data['context']['df'], question, qa_models, nb_squad_results=2
    )
    
    print(formatted_responses)

    return formatted_responses, new_answer(question, data, qa_models)

In [ ]:
##Test FAQ dialog system's part

In [ ]:
import tempfile
from unittest.mock import patch
from shutil import copyfile
from collections import defaultdict

def mock_faq_files(tmpdirname, faqs, faq_dic):

    faq_dic['path'] = path.join(tmpdirname, 'temp_faq.csv')
    faq_dic['config'] = path.join(tmpdirname, 'temp_config_faq.json')
    faq_dic['df'] = pd.DataFrame(faqs)
    faq_dic['df'].to_csv(faq_dic['path'], index=False)

    copyfile(configs.faq.tfidf_logreg_en_faq, faq_dic['config'])

    updates_faq_config_file(
        configs_path=faq_dic['config'],
        dataset_reader={'data_path': faq_dic['path']}
    )


def mock_context_file(tmpdirname, contexts, context_dic):

    context_dic['path'] = path.join(tmpdirname, 'temp_context.csv')
    context_dic['df'] = pd.DataFrame(contexts)
    context_dic['df'].to_csv(context_dic['path'], index=False)


@patch('__main__.get_input')
def test_context_response_with_no_updates(mock_input):
    mock_input.side_effect = ['Who is Enrique Jimenez?', 'N']
    data = {'context': defaultdict(str), 'faq': defaultdict(str)}
    contexts = {
        'context':
            [
                'Intekglobal has its headquarters located in TJ',
                'In Intekglobal we care about you',
                '''Enrique Jimenez is one of the smartest minds on the planet, 
                   he currently works as Intekglobal employee'''
            ],
        'topic': ['headquarters', 'mission', 'Enrique\'s biography']
    }

    faqs = {
        'Question':
            ['Minimum number of questions?', 'This is the other question?'],
        'Answer': ['Two', 'yes']
    }

    with tempfile.TemporaryDirectory() as tmpdirname:

        mock_faq_files(tmpdirname, faqs, data['faq'])
        mock_context_file(tmpdirname, contexts, data['context'])

        qa_models = load_qa_models(
            config_tfidf=data['faq']['config'], download=False
        )

        responses, status = question_response(data, qa_models)
        logging.debug(f'  {status}')
        logging.debug(f'  {responses}')
        assert 'no data updates..' == status
        assert 'one of the smartest minds on the planet' in responses

@patch('__main__.get_input')
def test_updating_faq_dataset(mock_input):

    new_answer = 'Intekglobal is one of the best companies in the world'
    mock_input.side_effect = ['What is Intekglobal?', 'Y', 'N', new_answer]

    data = {'context': defaultdict(str), 'faq': defaultdict(str)}
    contexts = {
        'context':
            [
                '''Tesla do important AI research. 
                The goal is to make an automatic pilot.
                '''
            ],
        'topic': ['Tesla AI']
    }

    faqs = {
        'Question':
            ['Who  owns Tesla Company?', 'Is this is heaven?'],
        'Answer': ['Elon Musk is the owner of Tesla', 'No, it is life on earth']
    }

    with tempfile.TemporaryDirectory() as tmpdirname:

        mock_faq_files(tmpdirname, faqs, data['faq'])
        mock_context_file(tmpdirname, contexts, data['context'])

        qa_models = load_qa_models(
            config_tfidf=data['faq']['config'], download=False
        )

        responses, status = question_response(data, qa_models)
        
        logging.debug(f'  {status}')
        logging.debug(f'  {responses}')
        
        assert 'FAQ dataset and model updated..' == status

        updated_faq = pd.read_csv(data['faq']['path'])

        assert updated_faq[updated_faq['Answer'] == new_answer
                              ].shape[0] == 1

        
test_context_response_with_no_updates()
test_updating_faq_dataset()

In [ ]:
#tests
import tempfile
from unittest.mock import patch
from shutil import copyfile
from collections import defaultdict

def mock_faq_files(tmpdirname, faqs, faq_dic):

    faq_dic['path'] = path.join(tmpdirname, 'temp_faq.csv')
    faq_dic['config'] = path.join(tmpdirname, 'temp_config_faq.json')
    faq_dic['df'] = pd.DataFrame(faqs)
    faq_dic['df'].to_csv(faq_dic['path'], index=False)

    copyfile(configs.faq.tfidf_logreg_en_faq, faq_dic['config'])

    updates_faq_config_file(
        configs_path=faq_dic['config'],
        dataset_reader={'data_path': faq_dic['path']}
    )


def mock_context_file(tmpdirname, contexts, context_dic):

    context_dic['path'] = path.join(tmpdirname, 'temp_context.csv')
    context_dic['df'] = pd.DataFrame(contexts)
    context_dic['df'].to_csv(context_dic['path'], index=False)

@patch('__main__.get_input')
def test_updated_faq_response(mock_input):

    new_answer = 'Intekglobal is one of the best companies in the world'
    question = 'What is Intekglobal?'
    mock_input.side_effect = [question, 'Y', 'N', new_answer, question, 'N']

    data = {'context': defaultdict(str), 'faq': defaultdict(str)}
    contexts = {
        'context':
            [
                '''One of the greatest punk rock bands from all the time
                is the Ramones.
                '''
            ],
        'topic': ['Ramones']
    }

    faqs = {
        'Question':
            ['Who was the leading vocal artist of the Ramones?', 'Is this is heaven?'],
        'Answer': ['Joey Ramone', 'No, it is life on earth']
    }

    with tempfile.TemporaryDirectory() as tmpdirname:

        mock_faq_files(tmpdirname, faqs, data['faq'])
        mock_context_file(tmpdirname, contexts, data['context'])

        qa_models = load_qa_models(
            config_tfidf=data['faq']['config'], download=False
        )
        old_responses, old_status = question_response(data, qa_models)
        new_responses, new_status = question_response(data, qa_models)

        logging.debug(f'Old response:\n {old_responses}')
        logging.debug(f'New response:\n{new_responses}')
        
        new_answer not in old_responses
        new_answer in new_responses

test_updated_faq_response()

In [ ]:
# Test Contex Dialog System's part

In [ ]:
import tempfile
from unittest.mock import patch
from shutil import copyfile
from collections import defaultdict


def mock_faq_files(tmpdirname, faqs, faq_dic):

    faq_dic['path'] = path.join(tmpdirname, 'temp_faq.csv')
    faq_dic['config'] = path.join(tmpdirname, 'temp_config_faq.json')
    faq_dic['df'] = pd.DataFrame(faqs)
    faq_dic['df'].to_csv(faq_dic['path'], index=False)

    copyfile(configs.faq.tfidf_logreg_en_faq, faq_dic['config'])

    updates_faq_config_file(
        configs_path=faq_dic['config'],
        dataset_reader={'data_path': faq_dic['path']}
    )


def mock_context_file(tmpdirname, contexts, context_dic):

    context_dic['path'] = path.join(tmpdirname, 'temp_context.csv')
    context_dic['df'] = pd.DataFrame(contexts)
    context_dic['df'].to_csv(context_dic['path'], index=False)


@patch('__main__.get_input')
def test_updated_context_response(mock_input):
    data = {'context': defaultdict(str), 'faq': defaultdict(str)}

    contexts = {
        'context': ['Intekglobal has its headquarters located in TJ', ],
        'topic': ['headquarters']
    }

    faqs = {
        'Question':
            ['Minimum number of questions?', 'This is the other question?'],
        'Answer': ['Two', 'yes']
    }

    question = 'What is a Chatbot?'
    new_topic = 'AI Tool & Chatbot Development'
    new_context = '''

A chatbot is an important tool for simulating intelligent conversations with humans.
Intekglobal chatbots efficiently live message on platforms such as Facebook Messenger, 
Slack, and Telegram. Assisting consumers with a variety of purposes and industries. 

But chatbots are more than just a cool technology advancement. They actually transform the user experience.
People want simple and convenient interactions with interface and products.

'''

    mock_input.side_effect = [
        question, 'YES', 'yes', new_topic, new_context, question, 'N'
    ]

    with tempfile.TemporaryDirectory() as tmpdirname:

        mock_faq_files(tmpdirname, faqs, data['faq'])
        mock_context_file(tmpdirname, contexts, data['context'])

        qa_models = load_qa_models(
            config_tfidf=data['faq']['config'], download=False
        )

        old_responses, old_status = question_response(data, qa_models)
        new_responses, new_status = question_response(data, qa_models)

        logging.info(f'Old status: {old_status}')
        logging.info(f'Old response:\n {old_responses}')
        logging.info(f'New response:\n{new_responses}')

        updated_context = pd.read_csv(data['context']['path'])

        assert updated_context[updated_context['context'] == new_context
                              ].shape[0] == 1

        assert updated_context[updated_context['topic'] == new_topic
                              ].shape[0] == 1

        assert 'an important tool for simulating intelligent conversations with humans' not in old_responses
        assert 'an important tool for simulating intelligent conversations with humans' in new_responses


test_updated_context_response()

In [ ]:
#export
def set_minimal_faq_questions(data):
    if data['df'].shape[0] > 1:
        return
    minimal_questions = [
        'Is this the Intekglobal Dialog System?',
        'What is the purpose of these two automated questions?'
    ]
    minimal_answers = [
        'This is the Intekglobal Dialog System', 'To populate the FAQ data file'
    ]
    minimal_faqs_df = pd.DataFrame(
        {
            'Question': minimal_questions,
            'Answer': minimal_answers
        }
    )
    data['df'] = pd.concat([data['df'], minimal_faqs_df])
    data['df'].to_csv(data['path'], index=False)
    logging.info(f' File created at {data["path"]}')


def set_minimal_contexts(data):
    if data['df'].shape[0] > 0:
        return

    minimal_context_df = pd.DataFrame({'topic': [], 'context': []})
    data['df'] = minimal_context_df
    data['df'].to_csv(data['path'], index=False)

    logging.info(f' File created at {data["path"]}')


def set_data_dict(file, data, question_type, data_dir):

    data['path'] = file if file is not None else path.join(
        data_dir, question_type + '_data.csv'
    )

    data['df'] = pd.read_csv(data['path']) if path.isfile(data['path']
                                                         ) else pd.DataFrame()

    if question_type == 'faq':
        set_minimal_faq_questions(data)
    if question_type == 'context':
        set_minimal_contexts(data)


def load_and_prepare_data(context_data_file, faq_data_file, data, configs_faq):

    PARENT_DIR = popen('$PWD').read().strip()

    if faq_data_file or context_data_file is None:
        DATA_DIR = path.join(PARENT_DIR, 'data')

        if not path.isdir(DATA_DIR):
            mkdir(DATA_DIR)
            logging.info(f'Data directory created at {DATA_DIR}')

    if configs_faq is None:
        configs_faq = path.join(PARENT_DIR, 'config_faq.json')
        copyfile(configs.faq.tfidf_logreg_en_faq, configs_faq)

    data['faq']['config'] = configs_faq

    set_data_dict(
        file=faq_data_file,
        data=data['faq'],
        question_type='faq',
        data_dir=DATA_DIR
    )
    set_data_dict(
        file=context_data_file,
        data=data['context'],
        question_type='context',
        data_dir=DATA_DIR
    )

    updates_faq_config_file(
        configs_path=configs_faq,
        dataset_reader={'data_path': data['faq']['path']}
    )

In [ ]:
#tests
import tempfile
from collections import defaultdict
from shutil import rmtree
from unittest.mock import patch
from let_me_answer_for_you.dialog_system import updates_faq_config_file


def test_set_minimal_faqs_with_more_than_one_question():
    with tempfile.TemporaryDirectory() as tmpdirname:
        data_file = path.join(tmpdirname, 'tmp_data.csv')
        questions = ['a?', 'b?']
        answers = ['a', 'b']
        df = pd.DataFrame({'Question': questions, 'Answer': answers})
        df.to_csv(data_file, index=False)
        data = {'df': df, 'path': data_file}
        set_minimal_faq_questions(data)

        assert data['df'].shape[0] == 2


def test_set_minimal_faqs_with_less_than_two_questions():
    with tempfile.TemporaryDirectory() as tmpdirname:
        data_file = path.join(tmpdirname, 'tmp_data.csv')
        questions = ['a?']
        answers = ['a']
        df = pd.DataFrame({'Question': questions, 'Answer': answers})
        df.to_csv(data_file, index=False)
        data = {'df': df, 'path': data_file}

        assert data['df'].shape[0] == 1

        set_minimal_faq_questions(data)

        assert data['df'].shape[0] == 3
        assert any(
            data['df'].Question == 'Is this the Intekglobal Dialog System?'
        )


def test_set_minimal_contexts():
    with tempfile.TemporaryDirectory() as tmpdirname:
        data_file = path.join(tmpdirname, 'tmp_data.csv')
        data = {'df': pd.DataFrame(), 'path': data_file}
        set_minimal_contexts(data)
        assert path.isfile(data['path'])
        assert all(data['df'].columns == ['topic', 'context'])


def test_set_data_dict_no_file():
    with tempfile.TemporaryDirectory() as tmpdirname:
        data = {'context': defaultdict(str)}
        set_data_dict(
            file=None,
            data=data['context'],
            data_dir=tmpdirname,
            question_type='context'
        )
        logging.debug(data)
        assert path.isfile(data['context']['path'])


@patch('__main__.popen')
def test_load_and_prepare_data(mock_popen):
    with tempfile.TemporaryDirectory() as tmpdirname:
        mock_popen("$PWD").read().strip.side_effect = [tmpdirname]
        #tmp_config_file = path.join(tmpdirname, 'tmp_file.json')
        #copyfile(configs.faq.tfidf_logreg_en_faq, tmp_config_file)
        data = {'context': defaultdict(str), 'faq': defaultdict(str)}

        load_and_prepare_data(
            context_data_file=None,
            faq_data_file=None,
            data=data,
            configs_faq=None
        )
        data_dir = path.join(tmpdirname, 'data')
        config_file = path.join(tmpdirname,'config_faq.json')
        assert path.isdir(data_dir)
        assert path.isfile(config_file)


test_set_minimal_faqs_with_more_than_one_question()
test_set_minimal_faqs_with_less_than_two_questions()
test_set_minimal_contexts()
test_set_data_dict_no_file()
test_load_and_prepare_data()

In [ ]:
#export
def dialog_system(context_data_file=None, faq_data_file=None, configs_faq=None):
    '''
     Main Dialog System
    '''
    run_shell_installs()

    data = {'context': defaultdict(str), 'faq': defaultdict(str)}

    load_and_prepare_data(
        context_data_file=context_data_file,
        faq_data_file=faq_data_file,
        configs_faq=configs_faq,
        data=data
    )

    qa_models = load_qa_models(config_tfidf=data['faq']['config'])

    while True:
        try:
            responses, _ = question_response(data=data, qa_models=qa_models)
        except (KeyboardInterrupt, EOFError, SystemExit):
            logging.debug('Goodbye!')
            return 'Goodbye!'

In [ ]:
# test  dialog_system()

from unittest.mock import patch
from let_me_answer_for_you.dialog_system import load_qa_models, load_and_prepare_data
from let_me_answer_for_you.dialog_system import question_response,run_shell_installs

@patch('__main__.run_shell_installs')
@patch('__main__.load_qa_models')
@patch('__main__.load_and_prepare_data')
@patch('__main__.question_response')
def test_main_keyboard_interrupt(
    mock_question_response,
    mock_pd_read_csv,
    mock_load_qa_models,
    mock_run_shell_installs,
):
    mock_question_response.side_effect = [
        KeyboardInterrupt(), EOFError(),
        SystemExit()
    ]
    assert 'Goodbye!' == dialog_system()
    assert 'Goodbye!' == dialog_system()
    assert 'Goodbye!' == dialog_system()


test_main_keyboard_interrupt()